In [5]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from statistics import median
import pickle
import csv
import multiprocess
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("../")
from projects import project_list

In [6]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [7]:
folder_path = '../data/25_1_travis_data/'
write_path = 'datasets/'

In [8]:
for p in project_list:
    
    p_name = p.split('/')[1]
    data = pd.read_csv(folder_path + p)
    data = get_first_failures(data)
    sec = len(data)//10
    
    for i in range(9):
        sec_data = data.iloc[i*sec:i*sec+sec]
        b_ids = sec_data['tr_build_id'].tolist()
        file_name = write_path + p_name + '_' + str(i) + '.pkl'
        with open(file_name, 'wb') as wfile:
            pickle.dump(b_ids, wfile)
        
    
    i = i+1
    sec_data = data.iloc[i*sec:]
    b_ids = sec_data['tr_build_id'].tolist()
    file_name = write_path + p_name + '_' + str(i) + '.pkl'
    with open(file_name, 'wb') as wfile:
        pickle.dump(b_ids, wfile)
        

In [9]:
data

,Unnamed: 0,tr_build_id,gh_project_name,gh_is_pr,gh_pr_created_at,gh_pull_req_num,gh_lang,git_merged_with,git_branch,gh_num_commits_in_push,...,tr_log_num_test_suites_run,tr_log_num_test_suites_ok,tr_log_num_test_suites_failed,tr_log_tests_failed,tr_log_testduration,tr_log_buildduration,tr_original_commit,tr_duration,tr_status,tr_jobs
0,500135,11225347,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,7.82,NaN,a0e030167e77fbff1ac8323df477a338282b8f9a,261.0,passed,"[11225348, 11225349]"
1,500136,11225347,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,12.86,NaN,a0e030167e77fbff1ac8323df477a338282b8f9a,261.0,passed,"[11225348, 11225349]"
2,500137,11225357,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,8.06,NaN,a0e030167e77fbff1ac8323df477a338282b8f9a,271.0,passed,"[11225358, 11225359]"
3,500138,11225357,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,11.81,NaN,a0e030167e77fbff1ac8323df477a338282b8f9a,271.0,passed,"[11225358, 11225359]"
4,514120,11670799,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,7.24,NaN,1ecbef21a2afba3ca0f734dce5bbb16192170421,249.0,passed,"[11670800, 11670801]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46884,3800082,151701991,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,NaN,152.00,b4e884c2ff3bef182b045692b59c1aceae3cb892,6287.0,passed,"[151701992, 151701993, 151701994, 151701995, 1..."
46885,3800083,151701991,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,NaN,117.00,b4e884c2ff3bef182b045692b59c1aceae3cb892,6287.0,passed,"[151701992, 151701993, 151701994, 151701995, 1..."
46886,3800084,151701991,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,NaN,139.00,b4e884c2ff3bef182b045692b59c1aceae3cb892,6287.0,passed,"[151701992, 151701993, 151701994, 151701995, 1..."
46887,3800085,151701991,checkstyle/checkstyle,False,NaN,NaN,java,NaN,master,NaN,...,NaN,NaN,NaN,NaN,NaN,12.34,b4e884c2ff3bef182b045692b59c1aceae3cb892,6287.0,passed,"[151701992, 151701993, 151701994, 151701995, 1..."
